In [ ]:
import sys
sys.path.append('src')

In [2]:
import numpy as np
from bridges import Model

/home/jcw/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jcw/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jcw/anaconda3/envs/tf14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jcw/anaconda3/envs/tf14/lib/python3.7/site-packages/ten

In [3]:
m = Model()

Loading checkpoint checkpoint/run1/model-310052
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-310052


In [7]:
def get_perplexity(model, sentences=["\n"], batch_size=None, verbose=False):
    # save current batch_size
    batch_size_bak=None
    if batch_size is not None and self.batch_size != batch_size:
        batch_size_bak = self.batch_size
    model.check_batch_size(batch_size)
    
    tkns = model.encode(sentence)
    print(tkns)
    logits = model.get_logits(
        context_tokens=tkns, batch_size=model.batch_size
    )[:, :-1, :] # None to keep dims intact
    
    print("logits, shape:", logits.shape)
    
    # don't take the last one (predicting the token after our sentence)
    # normalizing logits for numerical stability (does not affect the result)
    mu = np.mean(logits, axis=-1, keepdims=True)
    lm = logits - mu
    le = np.exp(lm)
    logprobs = le / np.sum(le, axis=1, keepdims=True)
    
    print("mu, shape:", mu.shape)
    print("lm, shape:", lm.shape)
    print("le, shape:", le.shape)
    print("logprobs, shape:", logprobs.shape)
    
#     return logprobs, tkns

    # scores = np.nan_to_num(
    #     [logprobs[:, i, token] for i, token in enumerate(tkns[:-1])]
    # )
    scores = np.nan_to_num(
        [
            [logprobs[b, i, token] for i, token in enumerate(tkns[:-1])]
            for b in range(model.batch_size)
        ]
    )
    print("scores, shape:", scores.shape)
    perplexities = 2 ** (-np.mean(np.log2(scores), axis=-1))
    # restoring batch size 
    if batch_size_bak: model.check_batch_size(batch_size_bak)    
    return perplexities

In [10]:
snt = [
    "Ah, nous voilà bien",
    "Je me baladais dans le gris sans fond",
    "LE COMTE. Quel déshonneur !",
    "Ah."
]

In [12]:
get_perplexity(m, snt)

[list([10910, 11, 299, 516, 7608, 346, 24247, 275, 2013])
 list([40932, 502, 3652, 4763, 271, 288, 504, 443, 1036, 271, 38078, 16245])
 list([2538, 9440, 9328, 13, 2264, 417, 39073, 1477, 47476, 333, 5145])]


ValueError: setting an array element with a sequence.